In [2]:
#!pip install spacy-langdetect

     -------------------------------------- 998.1/998.1 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for langdetect: filename=langdetect-1.0.7-py3-none-any.whl size=993439 sha256=3cc6d52711d958e89c4c8aff5fa1ab9534bd210a987bcc73954d29297a8d83f1
  Stored in directory: c:\users\wei.liu\appdata\local\pip\cache\wheels\87\8c\9a\41c0647bd03b3e11ca6968d3638a4e6e764220adf2886270cb
Successfully built langdetect


In [3]:
#!python -m spacy download en_core_web_lg

     -------------------------------------- 587.7/587.7 MB 1.7 MB/s eta 0:00:00
[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [1]:
import pandas as pd
import numpy as np
import re
from bertopic import BERTopic
from bertopic.representation import KeyBERTInspired
from bertopic.representation import MaximalMarginalRelevance
from hdbscan import HDBSCAN
import hdbscan
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import v_measure_score
from umap import UMAP

import time
from unidecode import unidecode
import spacy
from spacy.language import Language
from spacy_langdetect import LanguageDetector

# Data Pre-processing

In [2]:
# raw data
raw_pre = pd.read_excel('data/raw/uncat_oi_pre.xlsx',parse_dates=['Date_Created','Date_Last_Updated', 'DateScheduled'])
raw_pst = pd.read_excel('data/raw/uncat_oi_pst.xlsx',parse_dates=['Date_Created','Date_Last_Updated', 'DateScheduled'])

print(raw_pre.shape)
print(raw_pst.shape)


(27945, 26)
(49627, 26)


In [3]:
print(raw_pre['Incident_Source'].value_counts())
print(raw_pst['Incident_Source'].value_counts())


Public API        11735
End-User pages     8402
CX Console         7333
Utilities           429
Web Console          41
End-User Pages        5
Name: Incident_Source, dtype: int64
Web Console       18541
Agent desktop     12107
Public API         7738
VBCS Web           4043
CX Console         3298
End-User Pages     1545
Utilities          1144
Email               565
Live Chat           503
Mobile               79
None                 63
Web                   1
Name: Incident_Source, dtype: int64


In [3]:
# data from consoles and utilities only
raw_csl_pre = raw_pre[raw_pre['Incident_Source'].isin(['CX Console', 'Utilities', 'Web Console'])]
raw_csl_pst = raw_pst[raw_pst['Incident_Source'].isin(['CX Console', 'Utilities', 'Web Console','Agent desktop','Live Chat'])]
# data from apps
raw_app_pre = raw_pre[raw_pre['Incident_Source'].isin(['End-User pages', 'Oracle Integration', 'Public API'])]
raw_app_pst = raw_pst[~raw_pst['Incident_Source'].isin(['CX Console', 'Utilities', 'Web Console','Agent desktop','Live Chat'])]

In [4]:
print(raw_csl_pre.shape)
print(raw_csl_pst.shape)
print(raw_app_pre.shape)
print(raw_app_pst.shape)

(7803, 26)
(35593, 26)
(20137, 26)
(14034, 26)


In [10]:
raw_app_pst[raw_app_pst['Year']==2023].shape

(4751, 26)

In [8]:
print(raw_pre.columns)
print(raw_pst.columns)


Index(['Incident_ID', 'Reference_#', 'Subject', 'Status', 'Incident_Source',
       'Category_ID', 'Service', 'Severity', 'District', 'Latitude',
       'Longitude', 'Date_Created', 'Date_Last_Updated', 'Department',
       'Escalated', 'Year', 'Quarter', 'Week', 'Total_Incidents', 'No_Of_Days',
       'Description', 'Language', 'HaulerID', 'DateScheduled', 'DateClosed',
       'UserDevice'],
      dtype='object')
Index(['Incident_ID', 'Reference_#', 'Subject', 'Status', 'Incident_Source',
       'Category_ID', 'Service', 'Severity', 'District', 'Latitude',
       'Longitude', 'Date_Created', 'Date_Last_Updated', 'Department',
       'Escalated', 'Year', 'Quarter', 'Week', 'Total_Incidents', 'No_Of_Days',
       'Description', 'Language', 'HaulerID', 'DateScheduled', 'DateClosed',
       'UserDevice'],
      dtype='object')


## Step1. data cleaning

In [6]:
def only_non_letters(name):
    '''function that finds out if the documents contains only non english letters '''
    import string
    char_set = string.ascii_letters
    return all((True if x not in char_set else False for x in name))

def clear_mctxt(txt, pre_txt, post_text):
    '''function that cleans machine translated docs'''
    txt = txt[len(pre_txt):txt.index(post_text)]
    return txt

def get_lang_detector(nlp, name):
    return LanguageDetector()

nlp = spacy.load("en_core_web_lg")
Language.factory("language_detector", func=get_lang_detector)
nlp.add_pipe('language_detector', last=True)


# data preprocessing pipeline
def pipeline(data, cols, min_len):
    '''
    data pre-processing pipeline:
    1.discard unneeded columns;
    2.discard null/empty value;
    3.discard descriptions with too little words
    4.discard descriptions with only non letters
    5.discard descriptions in non english languages
    6.replace some special charaters
    data: pd dataframe that need to process
    cols: list of columns to keep
    min_len: descriptions with words less than min_len will be discarded
    '''
    # keep only cols
    df = data[cols]
    # keep non null descriptions
    df = df[~df['Description'].isnull()]
    # make sure description are text
    df['Description'] = df['Description'].astype(str)
    # discard short descriptions
    df['len'] = df['Description'].apply(lambda x: len(str(x).split()))
    df = df[(df['len'] >= min_len)]
    # discard docs with only non letters
    df = df[~df['Description'].apply(only_non_letters)]
    # discard test docs
    df = df[~df['Description'].str.contains('TEST')]
    df = df[~((df['Description'].str.contains('test'))&(df['Description'].str.contains('delete')))]
    df = df[~((df['Description'].str.contains('test'))&(df['Description'].str.contains('ignore')))]
    # clean machine translated docs
    pre_txt = "(Machine translated text. Report translation errors at https://bit.ly/fll)  "
    post_txt = "  --Original text--"
    df[df['Description'].str.contains('\(Machine tran')]['Description']=\
    df[df['Description'].str.contains('\(Machine tran')]['Description'].apply(lambda x: clear_mctxt(x, pre_txt, post_txt))
    
    # discard languages other than en 
    
    df['lang'] = [nlp(i)._.language['language'] for i in df['Description']]
    df = df[(df['lang']!='es')&(df['lang']!='vi')]
    
    # process description field to correct characters
    for char in ["â\?","ã\?","\xa0ï¸\x8f",'Â\xa0', 'â\?¢', "ï¼", "ï»¿ï»¿"]:
        df["Description"] = df["Description"].replace(char, " ", regex=True)
    df["Description"] = df["Description"].replace("â\?\?", "'", regex=True)
    df["Description"] = df["Description"].replace("â\?\", "'", regex=True)
    df["Description"] = df["Description"].replace("â€™", "'", regex=True)
    df["Description"] = df["Description"].replace("Ã\©", "e", regex=True)
    df["Description"] = df["Description"].replace("Ã¡", "a", regex=True)
    df["Description"] = df["Description"].replace('Ã\xad', "i", regex=True)
    df["Description"] = df["Description"].replace("Ãº", "u", regex=True)
    return df

In [7]:
# clean data
cols = ['Incident_ID','Date_Created','Description','Service']
st = time.time()
df_app_pre = pipeline(raw_app_pre, cols=cols, min_len=2)
df_csl_pre = pipeline(raw_csl_pre, cols=cols, min_len=2)
df_app_pst = pipeline(raw_app_pst, cols=cols, min_len=2)
df_csl_pst = pipeline(raw_csl_pst, cols=cols, min_len=2)

elapsed = time.time()-st
print("Run time: " + time.strftime("%H:%M:%S.{}".format(str(elapsed % 1)[2:])[:11], time.gmtime(elapsed)))

Run time: 00:21:42.66


In [8]:
#Give semi labels to data
label_dict = {"Pothole":0, "Abandoned Vehicle":1, "Illegal Dumping":2, "Other Issues":-1}
for df in [df_app_pre, df_csl_pre, df_app_pst, df_csl_pst]:
    df['semi_label'] = df['Service'].apply(lambda x: label_dict[x])


In [9]:
# save data for modeling
df_app_pre.to_csv('data/clean/clean_uncat_app_pre.csv', index=False)
df_csl_pre.to_csv('data/clean/clean_uncat_csl_pre.csv', index=False)
df_app_pst.to_csv('data/clean/clean_uncat_app_pst.csv', index=False)
df_csl_pst.to_csv('data/clean/clean_uncat_csl_pst.csv', index=False)


In [10]:
# save data for modeling
df_1819.to_csv('data/clean/clean_uncat_pre.csv', index=False)
df_2223.to_csv('data/clean/clean_uncat_pst.csv', index=False)
